In [1]:
# Import system modules

import os
import arcpy
from arcpy.sa import *
arcpy.env.overwriteOutput = True

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [2]:
# create definitions for NDWI band math using raster calculator and reclassify to show only water

def sentinelNDWI(Band3,Band8,outImage,outImageClip):
    output_raster = (Band3-Band8)/(Band3+Band8)
    # output_raster = (Raster(output_raster1),[-1,0,0,1],[0, 1])                                       
    output_raster.save(outImage)
    arcpy.management.Clip(
    in_raster=outImage,
    rectangle="-76.1471461843248 37.1523459886121 -76.0833069397383 37.3195073802082",
    out_raster=outImageClip,
    in_template_dataset=Clip_Area,
    nodata_value="-1",
    clipping_geometry="ClippingGeometry",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

def reclassH2O(h2O_Image):
    out_water = arcpy.sa.Reclassify(in_raster=ndwiClipFullPath,reclass_field="VALUE", remap="-1 0.1 1;0.1 1 NODATA",missing_values="DATA")
    out_water.save(h2O_Image)

def vesselPolygons(h2O_Image, vessel_polygon):
    arcpy.conversion.RasterToPolygon(
        in_raster=h2O_Image,
        out_polygon_features=vessel_polygon,
        simplify="NO_SIMPLIFY",
        raster_field="Value",
        create_multipart_features="SINGLE_OUTER_PART",
        max_vertices_per_feature=None
    )

In [3]:
# Loop through the folders and get a list of SAFE directories that contain the Sentinel data
# for each directory find the Band 3 and Band 8 images

directory = r'C:\\Users\\keit8223\\Documents\\ArcGIS\\Projects\\Sentinel Anchorage Areas\Data'
outFolder = r'C:\\temp\\'

Clip_Area = r"C:\Users\keit8223\Documents\ArcGIS\Projects\Sentinel Anchorage Areas\Ancorage_Area.shp"

for folder in os.listdir(directory):
    f = os.path.join(directory, folder)
    if os.path.isdir(f):
        for root, dirs, files in os.walk(f):
            for file in files:
                # print(file)
                if file.endswith("B03.jp2"):
                    ndwiImage = str(file.split('_')[0]) + '_' + str(file.split('_')[1]) + '_NDWI.tif'
                    ndwiFullPath = outFolder + ndwiImage
                    ndwiClipImage = str(file.split('_')[0]) + '_' + str(file.split('_')[1]) + '_NDWI_Clip.tif'
                    ndwiClipFullPath = outFolder + ndwiClipImage
                    # print(ndwiImage)
                    # print(ndwiFullPath)
                    waterImage = str(file.split('_')[0]) + '_' + str(file.split('_')[1]) + '_h2O.tif'
                    waterFullPath = outFolder + waterImage
                    vesselPoly = str(file.split('_')[0]) + '_' + str(file.split('_')[1]) + '_vessel_poly.shp'
                    vesselPolyPath = outFolder + vesselPoly
                    vesselPoint = str(file.split('_')[0]) + '_' + str(file.split('_')[1]) + '_vessel_point.shp'
                    vesselPointPath = outFolder + vesselPoint
                    B03 = Raster(os.path.join(root, file))
                if file.endswith("B08.jp2"):
                    B08 = Raster(os.path.join(root, file))

                    sentinelNDWI(B03,B08,ndwiFullPath,ndwiClipFullPath)
                    reclassH2O(waterFullPath)
                    vesselPolygons(waterFullPath,vesselPolyPath)                

: 

: 